# Munge the Pachon/Gemini South DIMM files

## Imports and jupyter boilerplate

In [1]:
import os
import numpy as np
import pandas as pd
import astropy
import astropy.units as u
from astropy.coordinates import EarthLocation, get_sun, AltAz

## Configuration

In [2]:
data_dir = '../../data'

## Read the data files

Edison Bustos shared these files with me on google drive an 2019-10-07.

In [3]:
dimm_files = [os.path.join(data_dir, 'Pachon_DIMM._2004-2018.csv'),
              os.path.join(data_dir, 'Pachon_DIMM_2016-today.txt')]
dimm_colnames = ['day','timeofday','hr','fwhm','flux','strehlL','strehlR']
dimm_data = [pd.read_csv(fn, names=dimm_colnames, sep='\s+', skiprows=1, parse_dates={'time': ['day', 'timeofday']}).set_index('time').sort_index() 
             for fn in dimm_files]

These files overlap, and contain some duplicate data.
Combine them and remove the duplicates:

In [4]:
dimm = pd.concat([dimm_data[0].loc[:'2016-12-21'], dimm_data[1].loc['2016-12-21':]]).sort_index().reset_index().drop_duplicates(subset=['time']).set_index('time')

Declare the timezone for the time, and make sure the time is present both as a column and an index of the `pandas.DataFrame`, with different names (for convenience):

In [5]:
dimm = dimm.tz_localize('UTC')
dimm = dimm.reset_index().set_index('time', drop=False)
dimm.index.name = 't'
dimm.head()

,time,hr,fwhm,flux,strehlL,strehlR
t,,,,,,
2004-03-17 02:33:15+00:00,2004-03-17 02:33:15+00:00,4763,0.71,466228,0.0,0.0
2004-03-17 02:34:35+00:00,2004-03-17 02:34:35+00:00,4763,0.74,470195,0.0,0.0
2004-03-17 02:35:42+00:00,2004-03-17 02:35:42+00:00,4763,0.74,460268,0.0,0.0
2004-03-17 02:36:49+00:00,2004-03-17 02:36:49+00:00,4763,0.75,462758,0.0,0.0
2004-03-17 02:37:58+00:00,2004-03-17 02:37:58+00:00,4763,0.72,457338,0.0,0.0


## Create some useful derived columns

In [6]:
dimm['mjd'] = dimm.index.to_julian_date()-2400000.5
dimm['month'] = dimm.index.month
dimm['year'] = dimm.index.year
dimm['data_month'] = 12*(dimm.year - np.min(dimm.year)) + dimm.month

# night is like mjd, but shifts the rollover to be noon, mean local solar time
site_longitude_deg = EarthLocation.of_site('Cerro Pachon').lon.deg
site_night_shift = (180.0 + site_longitude_deg)/360.0
dimm['night'] = np.floor(dimm.mjd + site_night_shift).astype(int)
dimm['am'] = ((dimm.mjd + site_night_shift) % 1) > 0.5

In [7]:
dimm['strehl'] = 0.5*(dimm.strehlL + dimm.strehlR)
dimm['strehl_frac_diff'] = np.abs(dimm.strehlL - dimm.strehlR)/dimm.strehl

Calculate the Fried parameter `r0` following Tokovinin 2002 eqn 5:

In [8]:
dimm['r0'] = 0.98*5e-7/np.radians(dimm.fwhm/(60*60))
dimm['log_r0'] = np.log10(dimm.r0)

In [9]:
pachon = EarthLocation.of_site('Cerro Pachon')
def solar_zd(t):
    assert t.values.dtype.str[-3:-1] == 'ns'
    ap_times = astropy.time.Time(t.values.view('i8') * u.ns, format='unix')
    solar_gcrs = get_sun(ap_times)
    time_altaz_coords = AltAz(obstime=ap_times, location=pachon)
    solar_altaz = solar_gcrs.transform_to(time_altaz_coords)
    assert solar_altaz[0].alt.unit == u.degree
    zd = 90 - solar_altaz.alt.value
    return zd

dimm['solar_zd'] = solar_zd(dimm.index)

## Save the results to an HDF5 file

In [10]:
dimm.to_hdf(os.path.join(data_dir, 'Pachon_DIMM_to_2019-10-07.h5'), 'dimm', complevel=9)

In [11]:
dimm.head()

,time,hr,fwhm,flux,strehlL,strehlR,mjd,month,year,data_month,night,am,strehl,strehl_frac_diff,r0,log_r0,solar_zd
t,,,,,,,,,,,,,,,,,
2004-03-17 02:33:15+00:00,2004-03-17 02:33:15+00:00,4763,0.71,466228,0.0,0.0,53081.106424,3,2004,3,53081,False,0.0,NaN,0.142352,-0.846637,134.478719
2004-03-17 02:34:35+00:00,2004-03-17 02:34:35+00:00,4763,0.74,470195,0.0,0.0,53081.107350,3,2004,3,53081,False,0.0,NaN,0.136581,-0.864611,134.707136
2004-03-17 02:35:42+00:00,2004-03-17 02:35:42+00:00,4763,0.74,460268,0.0,0.0,53081.108125,3,2004,3,53081,False,0.0,NaN,0.136581,-0.864611,134.897625
2004-03-17 02:36:49+00:00,2004-03-17 02:36:49+00:00,4763,0.75,462758,0.0,0.0,53081.108900,3,2004,3,53081,False,0.0,NaN,0.134760,-0.870440,135.087365
2004-03-17 02:37:58+00:00,2004-03-17 02:37:58+00:00,4763,0.72,457338,0.0,0.0,53081.109699,3,2004,3,53081,False,0.0,NaN,0.140375,-0.852711,135.281977


In [12]:
dimm.tail()

,time,hr,fwhm,flux,strehlL,strehlR,mjd,month,year,data_month,night,am,strehl,strehl_frac_diff,r0,log_r0,solar_zd
t,,,,,,,,,,,,,,,,,
2019-10-07 09:27:01+00:00,2019-10-07 09:27:01+00:00,2294,0.546,16451,0.318,0.519,58763.393762,10,2019,190,58763,True,0.4185,0.480287,0.185109,-0.732571,100.877917
2019-10-07 09:28:04+00:00,2019-10-07 09:28:04+00:00,2294,0.574,15658,0.314,0.515,58763.394491,10,2019,190,58763,True,0.4145,0.484922,0.176080,-0.754291,100.656648
2019-10-07 09:29:07+00:00,2019-10-07 09:29:07+00:00,2294,0.481,16000,0.311,0.534,58763.395220,10,2019,190,58763,True,0.4225,0.527811,0.210124,-0.677524,100.435255
2019-10-07 09:30:10+00:00,2019-10-07 09:30:10+00:00,2294,0.528,16457,0.321,0.523,58763.395949,10,2019,190,58763,True,0.4220,0.478673,0.191420,-0.718013,100.213740
2019-10-07 09:31:07+00:00,2019-10-07 09:31:07+00:00,2294,0.478,16484,0.303,0.526,58763.396609,10,2019,190,58763,True,0.4145,0.537998,0.211443,-0.674807,100.013217
